In [25]:
from __future__ import print_function
import os
import re
from PIL import Image
import numpy as np

import sys
sys.path.append('/Users/jiajiechen/Desktop/project2-CSE6240/code')
import image_parser

import pandas as pd
import theano as th
import time

In [ ]:
# from matplotlib import pyplot as plt
# %matplotlib inline

# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import confusion_matrix
# from CSE6240_utilities import plot_confusion_matrix
# import matplotlib.pylab as pylab

# from scipy.stats import pearsonr

In [14]:
datapath="/Users/jiajiechen/Desktop/project2-CSE6240/Data"

In [15]:
animals=image_parser(folder="01-Animal",datapath=datapath,
                     verbose=False,min_rows=64,min_cols=64)

In [16]:
animals.read()

In [17]:
animals.images.shape

(1571, 4096)

In [60]:
class k_Sparse_AutoEncoder(object):
    """a very simple structure of autoencoder,
    from internet, modified it so as to make
    it k_sparse_autoencoder

    @author: Jiajie Chen
    @date: apr2, 2016"""
    def __init__(self, X, k):
        assert type(X) is np.ndarray
        assert len(X.shape)==2
        assert type(k) is int
        self.X=X
        self.n=X.shape[1]
        self.m=X.shape[0]
        self.k=k
        self.W=np.asarray(np.random.uniform(
                          low = -4 * np.sqrt(6. / self.n),
                          high = 4 * np.sqrt(6. / self.n),
                          size = (self.n, self.n)), dtype=np.float)
        
        self.b1=np.zeros(shape=(self.n,),
                         dtype=np.float)
        self.b2=np.zeros(shape=(self.n,),
                         dtype=np.float)
    
    def cost_entropy(self):
        pass
    
    def gradient(self):
        pass
    
    def train(self, n_epochs = 100, mini_batch_size=1, learning_rate=0.1):
        hidden = self.X.dot(self.W)+self.b1
#         hidden = np.select(condlist, choicelist)
        # add a line that enforce the sparsity

        output = hidden.dot(self.W.T)+self.b2
#         output = self.output_function(output)
        return hidden, output

In [61]:
test = k_Sparse_AutoEncoder(animals.images,k=20)

In [62]:
r = test.train()

In [64]:
r[0]

array([[  1.65226322e-314,  -2.37188701e-314,   3.64444669e-314, ...,
         -2.12909663e-315,  -1.30693477e-314,  -1.64014014e-314],
       [  4.65110428e+002,  -4.91916970e+002,   1.03897518e+003, ...,
         -6.70013049e+002,  -5.19838529e+002,   9.46230531e+002],
       [ -3.80567603e+001,  -5.09111367e+002,   8.40439784e+002, ...,
         -4.09503217e+002,  -2.44249114e+002,   3.85226575e+002],
       ..., 
       [  4.54048814e+002,  -4.56073306e+002,   1.59737711e+003, ...,
         -4.72450608e+002,  -8.58209631e+001,   1.24722399e+003],
       [ -1.90881343e+002,  -2.17459876e+002,   1.58174553e+003, ...,
          1.02326060e+002,  -6.73338500e+002,   1.37163547e+003],
       [  4.14053510e+001,  -4.60429557e+002,   1.23708603e+003, ...,
         -7.36046941e+002,   8.28162751e+001,   5.50516459e+002]])

In [65]:
r[1]

array([[  1.81705014e-313,   2.13480602e-313,  -2.31688421e-314, ...,
         -7.95122768e-314,   6.39486512e-313,   8.50238495e-313],
       [  5.05013904e+003,   6.39312021e+003,   3.15588407e+003, ...,
         -7.71529679e+002,   2.22945313e+003,   3.42717003e+003],
       [  6.94107407e+003,   8.29674733e+003,   5.81949637e+003, ...,
         -1.52562770e+003,   4.64764227e+003,   4.62435972e+003],
       ..., 
       [  7.97487080e+003,   8.67974764e+003,   6.07960424e+003, ...,
         -1.06184374e+003,   4.51436737e+003,   5.48421254e+003],
       [  4.42629599e+003,   2.65308202e+003,   2.48362833e+003, ...,
         -7.94470887e+002,   8.92205484e+003,   6.95463002e+003],
       [  5.42557615e+003,   5.32243237e+003,  -1.85843219e+002, ...,
         -3.26617653e+003,   5.70596437e+003,   9.28291704e+003]])

In [7]:
class k_Sparse_AutoEncoder(object):
    """a very simple structure of autoencoder,
       from internet, modified it so as to make
       it k_sparse_autoencoder
       
       @author: Jiajie Chen
       @date: apr2, 2016"""
    def __init__(self, X, hidden_size, activation_function,
                 output_function):

        #X is the data, an m x n numpy matrix
        assert type(X) is np.ndarray
        assert len(X.shape)==2
        self.X=X
        self.X=th.shared(name='X', value=np.asarray(self.X, 
                         dtype=th.config.floatX),borrow=True)
        #The config.floatX and borrow=True stuff is to get this to run
        #fast on the gpu.

        self.n = X.shape[1]
        self.m = X.shape[0]
        #Hidden_size is the number of neurons in the hidden layer, an int.
        assert type(hidden_size) is int
        assert hidden_size > 0
        
        self.hidden_size = hidden_size
        
        initial_W = np.asarray(np.random.uniform(
                 low = -4 * np.sqrt(6. / (self.hidden_size + self.n)),
                 high = 4 * np.sqrt(6. / (self.hidden_size + self.n)),
                 size = (self.n, self.hidden_size)), dtype=th.config.floatX)
        self.W = th.shared(value=initial_W, name='W', borrow=True)
        
        self.b1 = th.shared(name='b1', value=np.zeros(shape=(self.hidden_size,),
                            dtype=th.config.floatX),borrow=True)
        self.b2 = th.shared(name='b2', value=np.zeros(shape=(self.n,),
                            dtype=th.config.floatX),borrow=True)
        
        self.activation_function=activation_function
        self.output_function=output_function
                     
    def train(self, n_epochs = 100, mini_batch_size=1, learning_rate=0.1):
        index = th.tensor.lscalar()
        x = th.tensor.matrix('x')
        params = [self.W, self.b1, self.b2]
        hidden = self.activation_function(th.tensor.dot(x, self.W)+self.b1)
        hidden = np.select(condlist, choicelist)
        output = th.tensor.dot(hidden,th.tensor.transpose(self.W))+self.b2
        output = self.output_function(output)
         
        #Use cross-entropy loss.
        L = -th.tensor.sum(x*th.tensor.log(output) + (1-x)*th.tensor.log(1-output), axis=1)
        cost=L.mean()       
        updates=[]
         
        #Return gradient with respect to W, b1, b2.
        gparams = th.tensor.grad(cost,params)
         
        #Create a list of 2 tuples for updates.
        for param, gparam in zip(params, gparams):
            updates.append((param, param-learning_rate*gparam))
         
        #Train given a mini-batch of the data.
        train = th.function(inputs=[index], outputs=[cost], updates=updates,
                            givens={x:self.X[index:index + mini_batch_size,:]})

        start_time = time.time()
        for epoch in xrange(n_epochs):
            print("Epoch:",epoch)
            for row in xrange(0,self.m, mini_batch_size):
                train(row)
        end_time = time.time()
        print("Average time per epoch=", (end_time-start_time)/n_epochs)
                    
    def get_hidden(self,data):
        x = th.tensor.dmatrix('x')
        hidden = self.activation_function(th.tensor.dot(x,self.W)+self.b1)
        transformed_data = th.function(inputs=[x], outputs=[hidden])
        return transformed_data(data)
     
    def get_weights(self):
        return [self.W.get_value(), self.b1.get_value(), self.b2.get_value()]

In [9]:
activation_function=th.tensor.nnet.sigmoid
output_function=activation_function

ksA = A = k_Sparse_AutoEncoder(animals.images,20,
                               activation_function,output_function)

In [35]:
th.tensor.dot(ksA.X, ksA.W)

dot.0

In [31]:
th.tensor.nnet.ultra_fast_sigmoid(th.tensor.dot(ksA.X, ksA.W)+ksA.b1)

ultra_fast_sigmoid.0

In [63]:
animals.images = animals.images / 255.0

In [64]:
activation_function=th.tensor.nnet.sigmoid
output_function=activation_function
A = k_Sparse_AutoEncoder(animals.images,20,
                         activation_function,output_function)
A.train(n_epochs=2, mini_batch_size=20) # with 1000 hidden units for 20 epochs with minibatch size of 20
W = np.transpose(A.get_weights()[0])


print(A.get_hidden(animals.images)[0])

Epoch: 0
Epoch: 1
Average time per epoch= 0.406615972519
[[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]


In [ ]:
# # you may change the folders' name to your customization
# images = []
# image_names = []

# images, image_names = readImg("01-Animal", images, image_names, 0)
# images, image_names = readImg("02-Fungus", images, image_names, 1)
# images, image_names = readImg("03-Geological Formation", images, image_names, 2)
# images, image_names = readImg("04-Person", images, image_names, 3)
# images, image_names = readImg("05-Plant, flora, plant life", images, image_names, 4)
# images, image_names = readImg("06-Sport", images, image_names, 5)